# Convolution-type MKV-SDE

## Utility functions

In [ ]:
import scipy
import math
import numpy as np
import matplotlib.pyplot as plt
import time
from numpy import linalg as LA
from numpy import mean

# Recursive function to generate list of factorial values up to n
def factorial(n):
    if n == 1:
        return [1, 1]
    else:
        return factorial(n-1)+[math.factorial(n)]
    
# Function for computing alpha_k(x) functions
def fun_alpha(K, f):
    return lambda x : [math.pi**(1/4) * (1/2)**(i/2) * f[i]**(-1/2) * x**i * np.exp(-(x**2)/4) for i in range(K+1)] 

# Function returning the derivative of alpha_k(x) functions
def fun_alpha1(K, f): 
    return lambda x : [-math.pi**(1/4) * (1/2)**(i/2+1) * f[i]**(-1/2) * x**(i-1) * (x**2 - 2*i) * np.exp(-(x**2)/4) for i in range(K+1)] 

# Function returning normalized Hermite polynomial basis functions H_i(x)
def fun_H(K, f):
    return lambda x : [ (2**i)**(-1/2) * f[i]**(-1/2) * (math.pi)**(-1/4) * (scipy.special.hermite(i, monic=False)(x)) for i in range(K+1)]

# Function returning a combination of Hermite polynomial derivatives
def fun_phi1(K, f):
    return lambda x : [-(math.pi)**(-(1/4)) * x, math.sqrt(2)/(math.pi)**(1/4) * (1 - x**2)] + \
        [(2**i)**(-1/2) * f[i]**(-1/2) * (math.pi)**(-1/4) * ( (2 * i * scipy.special.hermite(i-1, monic=False)(x)) - \
                                                              x * (scipy.special.hermite(i, monic=False)(x)) ) for i in range(2,K+1)]

# Monte Carlo simulation for stochastic differential equations
def monte_carlo(sigma, T, N, M, K, f):
    h = T / N   # time step size
    alpha = fun_alpha(K, f)
    H = fun_H(K, f)
    X = np.random.normal(0, 1, M)   # initial random samples
    gamma = np.zeros((K+1, N+1))
    gamma[:,0] = mean( (np.array(H(X)) * np.exp(-((X ** 2)/2))), axis=1 )   # Initial gamma

    for i in range(N):
        W = np.random.normal(0, 1, M)   # Brownian motion increment
        X = X + ( np.dot(gamma[:,i], np.array(alpha(X))) ) * h + sigma * math.sqrt(h) * W   # Euler-Maruyama update for X
        gamma[:,i+1] = mean( (np.array(H(X)) * np.exp(-((X ** 2)/2))), axis=1 )   # Update gamma using new X samples

    return X, gamma 

# Function to construct basis matrices for the algorithm
def base(T, N, n, K, f, basis_type):
    g = np.ones(n+1)
    cc = np.linspace(0, T, N+1)  # time grid
    a = np.zeros((K+1, n+1)) 
    X = np.random.normal(0, 1, 1)

    if basis_type == 'canonical':
        # Canonical polynomial basis
        g = np.array([ cc ** i for i in range(n+1)]) 
        a[0,0] = (math.sqrt(2) * math.pi)**(-(1/4))
        return a, g

    elif basis_type == 'lagrange':
        # Lagrange polynomial basis using Chebyshev nodes
        l = [(0 + T)/2 + (T - 0)/2 * np.cos(((2 * i + 1)/ (2 * n + 2)) * math.pi) for i in range(n+1)]
        g = np.array([math.prod([((cc - l[j]) / (l[i] - l[j])) for j in range(n+1) if j!=i]) for i in range(n+1)])
        a[0,:] = (math.sqrt(2) * math.pi)**(-(1/4))
        return a, g 

    else:
        return 'err'
    
# Euler-Maruyama scheme for coupled stochastic processes
def euler(a, sigma, n, N, M, h, K, g, alpha, alpha1, H):
    Z = np.zeros((N+1, M))
    Ztilde = np.zeros((N+1, M))
    Ytilde = np.zeros((n+1, K+1, N+1, M))
    Tensor = np.zeros((M, K+1, n+1))

    # Initialize with random Brownian samples
    Z[0,:] = np.random.normal(0, 1, M) 
    Ztilde[0,:] = np.random.normal(0, 1, M)

    for i in range(N):
        c = np.dot(a, g[:,i])   # coefficient vector
        W = np.random.normal(0, 1, (2, M))   # two independent Brownian increments

        Z[i+1] = Z[i] + ( np.dot(c, np.array(alpha(Z[i])) ) ) * h + sigma * math.sqrt(h) * W[0]   # Euler step for Z

        # Update Ytilde tensor
        Ytilde[:,:,i+1,:] = Ytilde[:,:,i,:] + \
            ( Tensor.transpose() * ( np.array(alpha(Ztilde[i])) * np.ones((n+1, K+1, M)) ) + \
             ( (K+1) * mean( ( c * np.ones((M, 1)) ).transpose() * np.array(alpha1(Ztilde[i])), axis=0 ) ) * Ytilde[:,:,i,:] ) * h

        # Euler step for auxiliary process Ztilde
        Ztilde[i+1] = Ztilde[i] + ( np.dot(c, np.array(alpha(Ztilde[i]))) ) * h + sigma * math.sqrt(h) * W[1]

    return Z, Ztilde, Ytilde

# Stochastic gradient descent algorithm
def stochastic_gradient_descent(a_0, n, r0, rho, sigma, N, M, K, eps, h, g, gamma, alpha, alpha1, H, phi1):

    a = a_0 
    norm = LA.norm(gamma)

    for m in range(5000):
        # Stop condition when relative error is below threshold
        if (LA.norm( np.dot(a,g) - gamma ) / norm < eps) :
            break

        eta = r0 / ((m + 1) ** rho)  # adaptive learning rate
        # Simulate stochastic dynamics
        Z, Ztilde, Ytilde = euler(a, sigma, n, N, M, h, K, g, alpha, alpha1, H) 

        # Compute difference between model and target
        term1 = ( np.array(H(Z)) * np.exp(-((Z ** 2)/2)) ) - ( np.array([(np.dot(a[i], g) * np.ones((M,1))).transpose() for i in range(K+1)]) )

        term2 = np.zeros((K+1, K+1, N+1, M))
        Tensor = np.zeros((M, N+1, K+1, K+1))
        Tensor[:,:,:,:] = np.eye(K+1)
        Tensor = Tensor.transpose()
        v = np.zeros((K+1, n+1))

        # Gradient computation over each basis index
        for j in range(n+1):
            term2[:,:,:,:] = ( np.array(phi1(Ztilde)) * np.exp(-((Ztilde ** 2)/2)) ) 
            term2 = term2 * Ytilde[j,:,:,:]
            term3 = np.swapaxes(term2,0,1) - (g[j,:] * np.ones((M, 1))).transpose() * Tensor
            # Gradient estimate using Monte Carlo averaging
            v[:,j] =  mean( 2 * h * (N+1) * mean ( (K+1) * mean(term1 * term3, axis = 1), axis = 1 ) , axis = 1)

        # Update coefficients using stochastic gradient descent
        a = a - eta * v

    return a, m


## Plots

### Monte Carlo densities

In [ ]:
# Parameters 

T = 1
N1 = 100
K = 10
sigma = 0.1
n = 3 
M1 = 10**3
h = 0.01 

x = np.linspace(-3, 4, 100)
fig = plt.figure() 
plt.xlabel("x") 
    
for K in [3, 5, 10, 20]:

    f = factorial(K)
    H = fun_H(K, f)

    # Euler - Monte Carlo
    
    start = time.process_time() 
    X, Gamma = monte_carlo(sigma, T, N1, M1, K, factorial(K))
    end = time.process_time()  
    densityMC = np.dot( Gamma[:,-1].transpose(), (np.array(H(x)) * np.exp(-((x ** 2)/2))) )
    plt.plot(x, densityMC, label='$\widetilde{w}^{(K),MC}_T$')

plt.legend()
# plt.savefig("DensitiesMC.pdf")

### Stochastic Gradient Descent densities

In [ ]:
# Parameters 

T = 1
N1 = 100
N = 100
K = 10
sigma = 0.1
n = 3 
M1 = 10**1
M = 100
h = T / N  
r0 = 5  
rho = 0.9 
eps = 0.01


x = np.linspace(-3, 4, 100)
fig = plt.figure() 
plt.xlabel("x") 

f = factorial(K)
alpha = fun_alpha(K, f)
alpha1 = fun_alpha1(K, f)
H = fun_H(K, f)
phi1 = fun_phi1(K, f)
a_0, g = base(T, N, n, K, f, 'lagrange')

# Euler - Monte Carlo

start = time.process_time() 
X, Gamma = monte_carlo(sigma, T, N1, M1, K, factorial(K))
end = time.process_time()  
densityMC = np.dot( Gamma[:,-1].transpose(), (np.array(H(x)) * np.exp(-((x ** 2)/2))) )
plt.plot(x, densityMC, label='$\widetilde{w}^{(K),MC}_T$')

# Stochastic Gradient Descent

start = time.process_time()
a, m = stochastic_gradient_descent(a_0, n, r0, rho, sigma, N, M, K, eps, h, g, Gamma, alpha, alpha1, H, phi1) 
end = time.process_time() 
print("SGD execution time: "+ str(end - start))
print(" ")
print("SGD steps: "+str(m))
print(" ")
densitySGD = np.dot( np.dot(a, g)[:,-1].transpose(), (np.array(H(x)) * np.exp(-((x ** 2)/2))) ) 
plt.plot(x, densitySGD, '--',  label='$\widetilde{w}^{(K),SGD}_T$')
plt.legend()
# err = LA.norm(densitySGD - densityBM)/LA.norm(densityBM)
# print('Err rel density K = '+str(K)+' : '+str(err))

# plt.savefig("DensitiesSGD.pdf")
plt.show()
